# Items demand utility

Import all the needed modules

In [1]:
import pandas as pd
from pandas import ExcelWriter
import numpy as np

Import from the statistics file the columns needed

In [2]:
excel_file = 'R12.xlsx'
df = pd.read_excel(excel_file, usecols = ['Item', 'Year month',
                                          'Sold qty', 'Used qty',
                                          'Not rep usage'])

Import from the master file the columns needed

In [3]:
master_file = 'Item master.xlsx'
df_master = pd.read_excel(master_file, usecols = ['Item no', 'Item name',
                                            'Product group descr'])

Import supply data for suppliers

In [4]:
excel_file = 'Supply.xlsx'
df_supply = pd.read_excel(excel_file, usecols = ['Item no', 'Supplier'])
df_supply.drop_duplicates(inplace=True)

Get the first word of the item column using str.split and str.get function

In [5]:
df['Item no'] = df['Item'].str.split().str.get(0)

Remove possible whitespaces in the figures

In [6]:
columns = ['Sold qty', 'Used qty', 'Not rep usage']

for column in columns:
    df[column] = df[column].astype(str)
    df[column] = df[column].str.replace(' ', '')
    df[column] = df[column].astype(float)

Get the demand

In [7]:
df['Demand'] = df['Sold qty'] + df['Used qty'] + np.where(df['Not rep usage'] > 0, df['Not rep usage'], 0)

Get the pivot table of the demand and year month and replace NaN with zeros

In [8]:
pivot_df = pd.pivot_table(df, values = 'Demand', index = 'Item no' , columns = 'Year month', aggfunc=np.sum)
pivot_df = pivot_df.fillna(0)
pivot_df = pd.DataFrame(pivot_df.to_records())
pivot_df.head()

,Item no,1906,1907,1908,1909,1910,1911,1912,2001,2002,2003,2004,2005
0,0645000,66.0,33.0,99.0,0.0,99.0,297.0,99.0,198.0,33.0,99.0,99.0,198.0
1,0645001,117.0,195.0,390.0,39.0,78.0,39.0,39.0,156.0,78.0,156.0,39.0,78.0
2,0645002,45.0,45.0,225.0,135.0,90.0,45.0,45.0,0.0,45.0,180.0,0.0,90.0
3,0645003,153.0,255.0,51.0,153.0,459.0,102.0,0.0,51.0,102.0,0.0,51.0,204.0
4,0645004,174.0,58.0,0.0,232.0,232.0,58.0,58.0,464.0,232.0,102.0,0.0,102.0


Total field for pivot_df

In [9]:
col_len = len(pivot_df.columns)
pivot_df['Total'] = pivot_df.iloc[:, 1:col_len].sum(axis=1, numeric_only=True)
pivot_df['Mean'] = pivot_df.iloc[:, 1:col_len].mean(axis=1, numeric_only=True)
pivot_df['StD'] = pivot_df.iloc[:, 1:col_len].std(axis=1, numeric_only=True) 
pivot_df['CV'] = pivot_df['StD'] / pivot_df['Mean']
pivot_df['Min'] = pivot_df.iloc[:, 1:col_len].min(axis=1, numeric_only=True)
pivot_df['Q25'] = pivot_df.iloc[:, 1:col_len].quantile(q=0.25, axis=1, numeric_only=True).round(0)
pivot_df['Median'] = pivot_df.iloc[:, 1:col_len].median(axis=1, numeric_only=True)
pivot_df['Q67'] = pivot_df.iloc[:, 1:col_len].quantile(q=0.67, axis=1, numeric_only=True).round(0)
pivot_df['Q75'] = pivot_df.iloc[:, 1:col_len].quantile(q=0.75, axis=1, numeric_only=True).round(0)
pivot_df['Max'] = pivot_df.iloc[:, 1:col_len].max(axis=1, numeric_only=True)
pivot_df.head()

,Item no,1906,1907,1908,1909,1910,1911,1912,2001,2002,...,Total,Mean,StD,CV,Min,Q25,Median,Q67,Q75,Max
0,0645000,66.0,33.0,99.0,0.0,99.0,297.0,99.0,198.0,33.0,...,1320.0,110.000000,83.642095,0.760383,0.0,58.0,99.0,99.0,124.0,297.0
1,0645001,117.0,195.0,390.0,39.0,78.0,39.0,39.0,156.0,78.0,...,1404.0,117.000000,101.154248,0.864566,39.0,39.0,78.0,131.0,156.0,390.0
2,0645002,45.0,45.0,225.0,135.0,90.0,45.0,45.0,0.0,45.0,...,945.0,78.750000,69.515368,0.882735,0.0,45.0,45.0,90.0,101.0,225.0
3,0645003,153.0,255.0,51.0,153.0,459.0,102.0,0.0,51.0,102.0,...,1581.0,131.750000,129.493506,0.982873,0.0,51.0,102.0,153.0,166.0,459.0
4,0645004,174.0,58.0,0.0,232.0,232.0,58.0,58.0,464.0,232.0,...,1712.0,142.666667,132.698793,0.930132,0.0,58.0,102.0,195.0,232.0,464.0


Get the 'Item name' for each item. Merge it with the master_df

In [10]:
df_master = pd.merge(df_master, df_supply, on='Item no') # Suppliers

# Possible format issues
df_master['Item no'] = df_master['Item no'].astype(str)
pivot_df['Item no'] = pivot_df['Item no'].astype(str)

demand_df = pd.merge(df_master, pivot_df, on='Item no')
demand_df = demand_df.fillna(0)

Output the demand to Excel

In [11]:
name = 'Items Demand.xlsx'
writer = pd.ExcelWriter(name)
demand_df.to_excel(writer, index = False)
writer.save()
print('Done bro!')

Done bro!
